## Note
> This code will not run as is, as the entire analysis requires certain actions that are outisde of a Python environment. It does however provide a very thorough framework for the data analysis process used for a vote tranposition process.

In [22]:
import pandas as pd
import geopandas as gpd
import numpy as np
import googlemaps
from shapely.geometry import Point
import matplotlib.pyplot as plt

#connect to Postgis...
import psycopg2
con = psycopg2.connect(dbname='boundary_redistribution', user='postgres', password='password')


import os
%matplotlib inline
from PIL import Image
import glob
from IPython.display import clear_output

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 900)

#prevent scientific notation on float data types
pd.options.display.float_format = '{:,.3f}'.format

## Explanation of the analysis
The objective of the analysis is to determine which party benefits most from the new electoral boundaries that were redrawn for 2018 going forward to 2028.<br>

In order to do so we ask the hypothetical question: If the 2018 boundaries were used in the 2016 and 2011 elections, how would the seat count in the Legislature change?<br>

## Load necessary files...
> All files in ESPG: 26914

In [23]:
%%time
#electoral districts
ed2008 = gpd.read_file(r'.\boundary_commission_mb\ed_geo_2011_2016\current_ed_2008.shp')
ed2018 = gpd.read_file(r'.\ed_geo_2011_2016\ed2018_official.shp')

#voting areas
va2011 = gpd.read_file(r'.\boundary_commission_mb\va_2011_2016\va_geo_2011.shp')
va2016 = gpd.read_file(r'.\boundary_commission_mb\va_2011_2016\va_geo_2016.shp')

#voting area results (also contains advanced and non-va specific results)
results = gpd.read_file(r'.\boundary_commission_mb\va_2011_2016\va_results_11_16_fix2.csv')

#historical results by electoral division
hist_results = gpd.read_file(r'.\boundary_commission_mb\ed_geo_2011_2016\mb_election_historical_summaries_by_ed.csv')

#Winnipeg and Manitoba properties....
wpg_parcels = gpd.read_file(r'.\boundary_commission_mb\properties_dwellings\wpg_roll.shp')
mb_parcels = gpd.read_file(r'.\boundary_commission_mb\properties_dwellings\mb_roll.shp')

#this is the comibnation of both above layers
master_prop = gpd.read_file(r'.\boundary_commission_mb\analysis_output\master_res_props.shp')

Wall time: 13.7 s


## Prepare results data
> (One row per va with party results across columns)

In [24]:
# VA results for ED splitting
results_box = results.loc[(results['va_num'] != 99) & (results['votes'].notna())]
results_box = results_box.pivot_table(index=['year','ed_num','va_num','ed_name'], columns='party', values='votes', aggfunc='sum')
results_box = results_box.reset_index()
results_box = results_box.sort_values(by=['year','ed_name','va_num'])
results_box['uidva'] = (results_box['year'].astype('str') + '-' + results_box['ed_num'].astype('str') + '-' + results_box['va_num'].astype('str'))

In [25]:
# VA results for ED splitting
results_nobox = results.loc[(results['va_num'] == 99) & (results['votes'].notna())]
results_nobox = results_nobox.pivot_table(index=['year','ed_num','va_num','ed_name'], columns='party', values='votes', aggfunc='sum')
results_nobox = results_nobox.reset_index()
results_nobox = results_nobox.sort_values(by=['year','ed_name','va_num'])
results_nobox['uided'] = (results_nobox['year'].astype('str') + '-' + results_nobox['ed_num'].astype('str'))

## Add required ID-type fields to map files for merging

In [26]:
# create new UIDva field for each VA map file
va2011['uidva'] = (va2011['year'].astype('str') + '-' + va2011['ed_num'].astype('str') + '-' + va2011['va_num'].astype('str'))
va2016['uidva'] = (va2016['year'].astype('str') + '-' + va2016['ed_num'].astype('str') + '-' + va2016['va_num'].astype('str'))

# create new UIDed field for each VA map file
va2011['uided'] = (va2011['year'].astype('str') + '-' + va2011['ed_num'].astype('str'))
va2016['uided'] = (va2016['year'].astype('str') + '-' + va2016['ed_num'].astype('str'))

# create new UID field for each ED map file
# because the ed2008 is used by 2011 and 2016 elections, must make two unique ID columns for joins
ed2008['uided11'] = ('2011' + '-' + ed2008['ed_num'].astype('str'))
ed2008['uided16'] = ('2016' + '-' + ed2008['ed_num'].astype('str'))

ed2018['uided11'] = ('2011' + '-' + ed2018['ed_num'].astype('str'))
ed2018['uided16'] = ('2016' + '-' + ed2018['ed_num'].astype('str'))

## Add original area fields to map files<br>
> VA files returns area for voting areas<br>
> ED files returns area for electorial division

In [27]:
va2011['area_orig'] = va2011['geometry'].area/10**6
va2016['area_orig'] = va2016['geometry'].area/10**6

ed2008['area_orig'] = ed2008['geometry'].area/10**6
ed2018['area_orig'] = ed2018['geometry'].area/10**6

## Merge results data with VA maps (box votes) and ED maps (no box votes)

In [28]:
# BOX votes: merge va2016 map with 2016 results using UIDva
va2016box = pd.merge(va2016, results_box, how='inner', on='uidva')
va2016box = va2016box.drop(columns=['id','year_y','va_num_y','ed_name_y', 'ed_num_y'])

# BOX votes: merge va2011 map with 2011 results using UIDva
va2011box = pd.merge(va2011, results_box, how='inner', on='uidva')
va2011box = va2011box.drop(columns=['id','year_y','va_num_y','ed_name_y','ed_num_y'])

# NO BOX votes: merge ed2008 map with 2016 results using UIDed
ed2008r16 = pd.merge(ed2008, results_nobox, how='inner', left_on='uided16', right_on='uided' )
ed2008r16 = ed2008r16.drop(columns=['year_y','va_num','ed_name_y', 'ed_num_y'])

# NO BOX votes: merge ed2008 map with 2011 results using UIDed
ed2008r11 = pd.merge(ed2008, results_nobox, how='inner', left_on='uided11', right_on='uided' )
ed2008r11 = ed2008r11.drop(columns=['year_y','va_num','ed_name_y', 'ed_num_y'])

## Prepare the master property layer for intersection operation
> - The file in its original format is too large for the processing power of the computer and so it needs to first be buffered and then dissolved in to a single multi-polygon.<br>
> - Execution time is long, so must save to local file before moving on.

In [29]:
%%time
# execution time avg: 21 secs
buf = master_prop['geometry'].buffer(50)

Wall time: 22.4 s


In [30]:
%%time
# execution time avg: 26 mins
crs = {'init': 'epsg:26914'}
buff = gpd.GeoDataFrame(geometry=[buf.unary_union],crs=crs)

Wall time: 24min 11s


In [31]:
%%time
# save dissolved file
buff.to_file(r'.\boundary_commission_mb\analysis_output\props_diss.shp',driver='ESRI Shapefile')

Wall time: 1min


In [32]:
# This is the final property file for intersections
buff = gpd.read_file(r'.\boundary_commission_mb\analysis_output\props_diss.shp')

## Output new spatial layers using the intersection of VA and ED maps with the Property map<br>
The logic behind this is that surface area in the province where no voters exist (i.e. where there are no dwellings) are to be excluded from the reassignment of votes math. The voting area that encompasses Churchill for example, the entire voting population lives in about 1% of the North-East corner of the territory, and so assigning votes the neighbouring electoral division could yield incorrect results.<br>

#### 2016 elections.....

In [33]:
%%time
# Execution time approx: 9mins
# Intersect va2016box with dissolved master parcel map
va2016box_prop = gpd.overlay(va2016box, buff, how='intersection')

Wall time: 7min 51s


In [34]:
# Execution time approx: 9mins
# Intersect ed2008r16 with dissolved master parcel map
ed2008r16_prop = gpd.overlay(ed2008r16, buff, how='intersection')

In [35]:
# add new area field now that non-resident footprints are eliminated
# this will be the basis of the vote distribution calculation afterwards for VAs
va2016box_prop['area_prop'] = va2016box_prop['geometry'].area/10**6

In [36]:
# add new area field now that non-resident footprints are eliminated
# this will be the basis of the vote distribution calculation afterwards for EDs
ed2008r16_prop['area_prop'] = ed2008r16_prop['geometry'].area/10**6

In [37]:
%%time
# save va2016box_prop to file
va2016box_prop.to_file(r'.\boundary_commission_mb\analysis_output\va2016box_prop.shp',driver='ESRI Shapefile')

Wall time: 7.95 s


In [38]:
%%time
# save ed2008r16_prop to file
ed2008r16_prop.to_file(r'.\boundary_commission_mb\analysis_output\ed2008r16_prop.shp',driver='ESRI Shapefile')

Wall time: 9.22 s


In [39]:
#### 2011 elections....

In [40]:
%%time
# Execution time approx: 9mins
# Intersect va2011box with dissolved master parcel map
va2011box_prop = gpd.overlay(va2011box, buff, how='intersection')

Wall time: 7min 59s


In [41]:
# Execution time approx: 9mins
# Intersect ed2008r11 with dissolved master parcel map
ed2008r11_prop = gpd.overlay(ed2008r11, buff, how='intersection')

In [42]:
# add new area field now that non-resident footprints are eliminated
# this will be the basis of the vote distribution calculation afterwards for VAs
va2011box_prop['area_prop'] = va2011box_prop['geometry'].area/10**6

In [43]:
# add new area field now that non-resident footprints are eliminated
# this will be the basis of the vote distribution calculation afterwards for EDs
ed2008r11_prop['area_prop'] = ed2008r11_prop['geometry'].area/10**6

In [44]:
%%time
# save va2011box_prop to file
va2011box_prop.to_file(r'.\boundary_commission_mb\analysis_output\va2011box_prop.shp',driver='ESRI Shapefile')

Wall time: 8.17 s


In [45]:
%%time
# save ed2008r11_prop to file
ed2008r11_prop.to_file(r'.\boundary_commission_mb\analysis_output\ed2008r11_prop.shp',driver='ESRI Shapefile')

Wall time: 9.48 s


## Intersection between VA- and ED- resident layers with ED2018 layer

#### 2016 elections....

In [46]:
%%time
# execution time approx: 
# read in final VA and ED resident layers
va2016box_prop = gpd.read_file(r'.\boundary_commission_mb\analysis_output\va2016box_prop.shp')
ed2008r16_prop = gpd.read_file(r'.\boundary_commission_mb\analysis_output\ed2008r16_prop.shp')

Wall time: 1.64 s


In [47]:
%%time
# execution time approx: 6:20 mins
# Intersect va2016box_prop with 2018 boundaries
va2016box_trans = gpd.overlay(va2016box_prop, ed2018, how='intersection')

Wall time: 2min 10s


In [48]:
%%time
# execution time approx: 3:40 mins
# Intersect ed2008r16_prop with 2018 boundaries
ed2008r16_trans = gpd.overlay(ed2008r16_prop, ed2018, how='intersection')

Wall time: 1min 22s


#### 2011 elections....

In [49]:
# read in final VA and ED resident layers
va2011box_prop = gpd.read_file(r'.\boundary_commission_mb\analysis_output\va2011box_prop.shp')
ed2008r11_prop = gpd.read_file(r'.\boundary_commission_mb\analysis_output\ed2008r11_prop.shp')

In [50]:
%%time
# execution time approx: 1:40 mins
# Intersect va2011box_prop with 2018 boundaries
va2011box_trans = gpd.overlay(va2011box_prop, ed2018, how='intersection')

Wall time: 37 s


In [51]:
%%time
# execution time approx: 1:15 mins
# Intersect ed2008r11_prop with 2018 boundaries
ed2008r11_trans = gpd.overlay(ed2008r11_prop, ed2018, how='intersection')

Wall time: 1min 21s


## Create new areas for new pieces and assign weighting for vote distribution

#### 2016 elections

In [52]:
# add new areas after cookie-cutter
va2016box_trans['area_new'] = va2016box_trans['geometry'].area/10**6
ed2008r16_trans['area_new'] = ed2008r16_trans['geometry'].area/10**6

In [53]:
va2016box_trans['weight'] = va2016box_trans['area_new']/va2016box_trans['area_prop']
ed2008r16_trans['weight'] = ed2008r16_trans['area_new']/ed2008r16_trans['area_prop']

#### 2011 elections

In [54]:
# add new areas after cookie-cutter
va2011box_trans['area_new'] = va2011box_trans['geometry'].area/10**6
ed2008r11_trans['area_new'] = ed2008r11_trans['geometry'].area/10**6

In [55]:
va2011box_trans['weight'] = va2011box_trans['area_new']/va2011box_trans['area_prop']
ed2008r11_trans['weight'] = ed2008r11_trans['area_new']/ed2008r11_trans['area_prop']

## Resort rows and rename certain columns

#### 2016 elections....

In [56]:
va2016final = va2016box_trans.sort_values(by=['year_x','ed_name_x','va_num_x'])
va2016final = va2016final.rename(columns={'ed_name_x':'ed_name_orig', 'area_orig_':'area_va_orig','area_ori_1':'area_va_orig', 'ed_name':'ed_name_new'})

In [57]:
va2016edfinal = ed2008r16_trans.sort_values(by=['year_x','ed_name_x'])
va2016edfinal = va2016edfinal.rename(columns={'ed_name_x':'ed_name_orig','area_orig_1':'area_ed_orig', 'ed_name':'ed_name_new'})

#### 2011 elections....

In [58]:
va2011final = va2011box_trans.sort_values(by=['year_x','ed_name_x','va_num_x'])
va2011final = va2011final.rename(columns={'ed_name_x':'ed_name_orig', 'area_orig_':'area_va_orig','area_ori_1':'area_va_orig', 'ed_name':'ed_name_new'})

In [59]:
va2011edfinal = ed2008r11_trans.sort_values(by=['year_x','ed_name_x'])
va2011edfinal = va2011edfinal.rename(columns={'ed_name_x':'ed_name_orig','area_orig_1':'area_ed_orig', 'ed_name':'ed_name_new'})

## Determine vote proportion to assign to intersection VA chunks

#### 2016 elections....

In [60]:
#weight votes for VA2016
va2016final['cpc-m_new'] = va2016final['cpc-m']*va2016final['weight']
va2016final['gpm_new'] = va2016final['gpm']*va2016final['weight']
va2016final['ind_new'] = va2016final['ind']*va2016final['weight']
va2016final['lib_new'] = va2016final['lib']*va2016final['weight']
va2016final['mp_new'] = va2016final['mp']*va2016final['weight']
va2016final['ndp_new'] = va2016final['ndp']*va2016final['weight']
va2016final['pc_new'] = va2016final['pc']*va2016final['weight']

In [61]:
#weight votes for ED2016
va2016edfinal['cpc-m_new'] = va2016edfinal['cpc-m']*va2016edfinal['weight']
va2016edfinal['gpm_new'] = va2016edfinal['gpm']*va2016edfinal['weight']
va2016edfinal['ind_new'] = va2016edfinal['ind']*va2016edfinal['weight']
va2016edfinal['lib_new'] = va2016edfinal['lib']*va2016edfinal['weight']
va2016edfinal['mp_new'] = va2016edfinal['mp']*va2016edfinal['weight']
va2016edfinal['ndp_new'] = va2016edfinal['ndp']*va2016edfinal['weight']
va2016edfinal['pc_new'] = va2016edfinal['pc']*va2016edfinal['weight']

#### 2011 elections....

In [62]:
#weight votes for VA2011
va2011final['cpc-m_new'] = va2011final['cpc-m']*va2011final['weight']
va2011final['gpm_new'] = va2011final['gpm']*va2011final['weight']
va2011final['ind_new'] = va2011final['ind']*va2011final['weight']
va2011final['lib_new'] = va2011final['lib']*va2011final['weight']
va2011final['mp_new'] = va2011final['mp']*va2011final['weight']
va2011final['ndp_new'] = va2011final['ndp']*va2011final['weight']
va2011final['pc_new'] = va2011final['pc']*va2011final['weight']

In [63]:
#weight votes for ED2016
va2011edfinal['cpc-m_new'] = va2011edfinal['cpc-m']*va2011edfinal['weight']
va2011edfinal['gpm_new'] = va2011edfinal['gpm']*va2011edfinal['weight']
va2011edfinal['ind_new'] = va2011edfinal['ind']*va2011edfinal['weight']
va2011edfinal['lib_new'] = va2011edfinal['lib']*va2011edfinal['weight']
va2011edfinal['mp_new'] = va2011edfinal['mp']*va2011edfinal['weight']
va2011edfinal['ndp_new'] = va2011edfinal['ndp']*va2011edfinal['weight']
va2011edfinal['pc_new'] = va2011edfinal['pc']*va2011edfinal['weight']

## Narrow down to finals columns for groupby newly assigned ED

#### 2016 elections....

In [64]:
va2016final__ = va2016final.iloc[:,[20,28,29,30,31,32,33,34]]

In [65]:
va2016edfinal__ = va2016edfinal.iloc[:,[21,29,30,31,32,33,34,35]]

In [66]:
va2016final__grouped = va2016final__.groupby('ed_name_new').sum()

In [67]:
va2016edfinal__grouped = va2016edfinal__.groupby('ed_name_new').sum()

#### 2011 elections....

In [68]:
va2011final__ = va2011final.iloc[:,[20,28,29,30,31,32,33,34]]

In [69]:
va2011edfinal__ = va2011edfinal.iloc[:,[21,29,30,31,32,33,34,35]]

In [70]:
va2011final__grouped = va2011final__.groupby('ed_name_new').sum()

In [71]:
va2011edfinal__grouped = va2011edfinal__.groupby('ed_name_new').sum()

## Sum both group by outcomes for VA and ED

#### 2016 elections....

In [72]:
hypothetical2016_results = va2016final__grouped.add(va2016edfinal__grouped)

#### 2011 elections....

In [73]:
hypothetical2011_results = va2011final__grouped.add(va2011edfinal__grouped)

## Get max of columns and final seat count

#### 2016 elections....

In [74]:
hypothetical2016_outcomes = hypothetical2016_results.idxmax(axis=1)

In [75]:
hypothetical2016_outcomes.value_counts()

pc_new     41
ndp_new    13
lib_new     3
dtype: int64

## Conclusion: What are the hypothetical race outcomes using 2016 results?

In [80]:
hypothetical2016_results

,cpc-m_new,gpm_new,ind_new,lib_new,mp_new,ndp_new,pc_new
ed_name_new,,,,,,,
Agassiz,0.000,381.208,839.874,67.031,19.504,453.474,"5,368.396"
Assiniboia,0.000,669.706,0.000,"1,697.377",0.000,"2,578.034","4,065.074"
Borderland,0.000,152.456,0.000,"1,144.181",0.000,289.587,"6,077.819"
Brandon East,0.000,0.000,0.000,765.497,0.000,"2,302.017","3,259.293"
Brandon West,0.000,0.000,0.000,516.871,0.126,"1,625.396","4,586.527"
Burrows,25.714,253.029,0.000,"2,464.314",2.998,"2,003.489","1,175.117"
Concordia,0.000,0.017,0.000,692.388,271.859,"2,971.820","2,809.955"
Dauphin,0.000,609.193,70.108,706.145,140.255,"2,410.380","5,169.242"
Dawson Trail,0.000,214.919,0.000,"1,481.179",382.458,"1,512.222","4,242.041"


## Conclusion: Who benefits most?
For the 2016 elections....
> Progressive Conservatives gain 1 seat (from 40: to 41);<br>
> New Democratic Party loses 1 seat (from 14: to 13);<br>
> Liberals Party maintains seats (from 3: to 3)<br>